In [12]:
import mdtraj as md
import torch
from omm import ProteinImplicit
from tqdm import tqdm
import h5py
import openmm
import numpy as np

In [10]:
def compute_energy_from_traj(traj: md.Trajectory, amber_filename:str, num_relax_steps:int=0, temperature:float=300):
    # Returns potential and kinetic energy for each frame in the trajectory
    # assumes kT units with T = 300 Kelvin unless otherwise specified
    
    simulation_args = {"temperature": temperature, "temperature_units": "kelvin", "friction": 100.0, "dt": 0.00002, "time_units": "picoseconds", "prior_weight": None, 
                    "integrator_to_use": "overdamped", "do_energy_minimization": False, "chk_freq": 10000000, "device": "OpenCL", "fix":"backbone"}
    solvent_args = {"implicit_solvent": "OBC2", "implicit_solvent_kappa": 0.1, "implicit_solvent_kappa_length_units": "nanometer"}

    p = ProteinImplicit(
        filename = amber_filename, chk=0,
        simulation_args=simulation_args,
        solvent_args=solvent_args,
        save_filename = f"./"
        )

    gen_positions = traj.xyz # mdtraj saves in nanometers by default
    all_pe = torch.zeros((len(gen_positions,)))
    all_ke = torch.zeros((len(gen_positions,)))

    for i, position in tqdm(enumerate(gen_positions), mininterval=10):
        pos, pe, ke, forces = p.relax_energies(10 * position, # convert to angstroms
                                    velocities=True,
                                    num_relax_steps=num_relax_steps, # 0 relax steps means no relaxation, just get energies
                                    length_units="angstroms",
                                    time_units="picoseconds",
                                    energy_units="kT")
        all_pe[i] = pe
        all_ke[i] = ke

    return all_pe, all_ke


In [ ]:
chignolin_topology = md.load_topology("/data2/scratch/group_scratch/cg/datasets/chignolin/chignolin.pdb")
frames_to_save = None  # Specify frames to save, or None to save all frames
with h5py.File("/data2/scratch/group_scratch/cg/datasets/chignolin/chignolin_traj_all.hdf5", "r") as file:
    if "positions" not in file.keys():
        raise ValueError("No positions in file")
    positions = file["positions"]
    length_units = file.attrs["length_units"]
    conversion_factor = getattr(openmm.unit, 
                                length_units).conversion_factor_to(openmm.unit.nanometer)
    if frames_to_save is None:
        frames_to_save = np.arange(positions.shape[0])
    trajectory = md.Trajectory(positions[frames_to_save, :, :] * conversion_factor,
                                chignolin_topology)

In [13]:
mini_traj = trajectory[:1000]  # Use first 1000 frames for minimization
pe, ke = compute_energy_from_traj(mini_traj, "/data2/scratch/group_scratch/cg/datasets/chignolin/chignolin")

/home/dunne/boltz-likelihoods/src/omm/omm.py:271: UserWarning: Check all Implicit solvent parameters (e.g. solvent)
  warnings.warn("Check all Implicit solvent parameters (e.g. solvent)")
1000it [00:02, 352.02it/s]
